In [1]:
!pip install transformers
!pip install datasets
!pip install soundfile
!pip install librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [3]:
from datasets import load_dataset
dataset=load_dataset("ashraq/esc50",split="train[0:10]")

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [4]:
audio_sample=dataset[0]

In [5]:
audio_sample

{'filename': '1-100032-A-0.wav',
 'fold': 1,
 'target': 0,
 'category': 'dog',
 'esc10': True,
 'src_file': 100032,
 'take': 'A',
 'audio': {'path': None,
  'array': array([0., 0., 0., ..., 0., 0., 0.]),
  'sampling_rate': 44100}}

In [ ]:
print(audio_sample)

In [8]:
from IPython.display import Audio as IPythonAudio

IPythonAudio(audio_sample["audio"]["array"],
             rate=audio_sample["audio"]["sampling_rate"])

In [9]:
from transformers import pipeline
zero_shot_classifier=pipeline(
    task="zero-shot-audio-classification",model="laion/clap-htsat-unfused")


config.json:   0%|          | 0.00/5.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/615M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

In [10]:
#sampling: measuring value of a continous signal at fixed time steps

In [11]:
#sampling rate: no of samples taken in one second
# 8,000hz: telephone/walkie-talkie
# 192,000hz: high-resolution audio


In [ ]:
#5 second sound at 8k hz is 40,000 signal values
#transformers are trained on data with same sampling rate.


In [12]:
1*192000#data of 192k hz

192000

In [13]:
192000/16000 #model trained at 16kz

12.0

In [15]:
zero_shot_classifier.feature_extractor.sampling_rate

48000

In [16]:
audio_sample["audio"]["sampling_rate"]

44100

In [17]:
from datasets import Audio
dataset=dataset.cast_column("audio",
                            Audio(sampling_rate=48_000))

In [19]:
audio_sample=dataset[0]
audio_sample

{'filename': '1-100032-A-0.wav',
 'fold': 1,
 'target': 0,
 'category': 'dog',
 'esc10': True,
 'src_file': 100032,
 'take': 'A',
 'audio': {'path': None,
  'array': array([0., 0., 0., ..., 0., 0., 0.]),
  'sampling_rate': 48000}}

In [20]:
candidate_labels=["Sound of a dog","sound of a vacumn cleaner"]
zero_shot_classifier(audio_sample["audio"]["array"],
                     candidate_labels=candidate_labels)

[{'score': 0.9987805485725403, 'label': 'Sound of a dog'},
 {'score': 0.0012194284936413169, 'label': 'sound of a vacumn cleaner'}]

In [22]:
candidate_labels=[    "Sound of a child crying",  "Sound of vacuum cleaner",  "Sound of a bird singing","Sound of an airplane"]

In [24]:
zero_shot_classifier(audio_sample["audio"]["array"],
                     candidate_labels=candidate_labels)

[{'score': 0.6172533631324768, 'label': 'Sound of a bird singing'},
 {'score': 0.21602550148963928, 'label': 'Sound of vacuum cleaner'},
 {'score': 0.12547214329242706, 'label': 'Sound of an airplane'},
 {'score': 0.04124903678894043, 'label': 'Sound of a child crying'}]